<a href="https://colab.research.google.com/github/muzuvajoshua/KinderCare/blob/main/Mypipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries
import os
import cv2
import edt
import pickle
import pandas as pd
import numpy as np
import skimage
import imageio
from skimage import io
from skimage import exposure
from skimage import measure
from skimage import segmentation
from skimage import util
from scipy import ndimage as ndi
from scipy.ndimage import distance_transform_edt
import matplotlib.pyplot as plt
import seaborn as sns
import btrack
import zarr
from stardist.models import StarDist2D
from stardist import random_label_cmap, _draw_polygons
from csbdeep.utils import normalize
from tqdm import tqdm
from btrack.constants import BayesianUpdates
from btrack.io import localizations_to_objects

In [ ]:
# Set path and file names
path = '/content/mydrive/MyDrive/RedChannel_ERKreporter'
image_name = 'Red_C8_1.tif'
save_path = '/content/mydrive/MyDrive/RedChannel_ERKreporter/results'

# Create result folder
os.makedirs(save_path, exist_ok=True)


In [ ]:
# Define some parameters
nuc_channel = 1
ring_size = 2
show_preview = True
file_path = os.path.join(path, image_name)

In [ ]:
# Load image data
imERKl = imageio.imread(file_path)

# Check image shape
print('Size of the image: ', imERKl.shape)


<ipython-input-4-b8d3bfa41562> (2): Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


Size of the image:  (570, 1040, 1392)


In [ ]:
# Load StarDist2D model
modelStar = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
5320433/5320433 [==============================] - 0s 0us/step
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [ ]:
# Initialize lists for storing results
nuc_list = []
df = []

# Loop over time points in the image data
for i in tqdm(range(imERKl.shape[0])):
    img = imERKl[i, nuc_channel, ...]
    p1, p2 = np.percentile(img, (1, 99.8))
    img_norm = np.clip(img, p1, p2)
    img_norm = (img_norm - p1) / (p2 - p1)
    img_norm = np.expand_dims(img_norm, axis=0)

    # Use StarDist2D model to label nuclei
    label, detail = modelStar.predict_instances(img_norm, prob_thresh=0.47, show_tile_progress=False, verbose=False)

    if np.max(label) > 0:
        df_class = skimage.measure.regionprops_table(label, intensity_image=imERKl[i, nuc_channel, ...], properties=('label', 'centroid', 'area'))
        df_class = pd.DataFrame(df_class)
        df_class['t'] = i
        df_class['centroid-0'] = df_class['centroid-0'].astype('int')
        df_class['centroid-1'] = df_class['centroid-1'].astype('int')
        df_class.rename(columns={'centroid-0': 'y', 'centroid-1': 'x'}, inplace=True)

        nuc_list.append(label)
        df.append(df_class)

# Check if any objects were detected
if len(df) > 0:
    df = pd.concat(df)
    seg_stack = np.stack(nuc_list)
else:
    print("No instances detected.")

  0%|          | 0/570 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  0%|          | 1/570 [00:01<17:29,  1.85s/it]

1/1 [==============================] - 0s 72ms/step


  0%|          | 2/570 [00:01<08:01,  1.18it/s]

...

1/1 [==============================] - 0s 73ms/step


100%|█████████▉| 569/570 [01:46<00:00,  5.62it/s]

1/1 [==============================] - 0s 70ms/step


100%|██████████| 570/570 [01:47<00:00,  5.32it/s]

No instances detected.


In [ ]:
# Prepare data for tracking
objects_to_track = localizations_to_objects(df)


[INFO][2023/07/13 11:59:16 AM] Objects are of type: <class 'list'>
INFO:btrack.io.utils:Objects are of type: <class 'list'>


In [ ]:
# Run tracking
with btrack.BayesianTracker() as tracker:
    # Configure the tracker
    tracker.configure_from_file('/cell_config3_new.json')
    tracker.update_method = BayesianUpdates.EXACT
    tracker.max_search_radius = 10

    # Append the objects to be tracked
    tracker.append(objects_to_track)

    # Set the volume
    tracker.volume = ((0, imERKl.shape[2]), (0, imERKl.shape[1]))

    # Track objects
    tracker.track_interactive(step_size=100)

    # Optimize tracking
    tracker.optimize()

    # Export tracking data
    if not objects_to_track:
      print("No objects to track.")
    else:
      tracker.export(os.path.join(save_path, 'tracks.h5'), obj_type='obj_type_1')


    # Get the tracks as a list
    tracks = tracker.tracks


[INFO][2023/07/13 12:12:02 PM] Loaded btrack: /usr/local/lib/python3.10/dist-packages/btrack/libs/libtracker.so
INFO:btrack.libwrapper:Loaded btrack: /usr/local/lib/python3.10/dist-packages/btrack/libs/libtracker.so
[INFO][2023/07/13 12:12:02 PM] Starting BayesianTracker session
INFO:btrack.core:Starting BayesianTracker session
[INFO][2023/07/13 12:12:02 PM] Loading configuration file: /cell_config3_new.json
INFO:btrack.config:Loading configuration file: /cell_config3_new.json
[INFO][2023/07/13 12:12:02 PM] Objects are of type: <class 'list'>
INFO:btrack.io.utils:Objects are of type: <class 'list'>
[WARNING][2023/07/13 12:12:02 PM] `track_interactive` will be deprecated. Use `track` instead.
[INFO][2023/07/13 12:12:02 PM] Starting tracking... 
INFO:btrack.core:Starting tracking... 
[INFO][2023/07/13 12:12:02 PM] Update using: ['MOTION']
INFO:btrack.core:Update using: ['MOTION']
[ERROR][2023/07/13 12:12:02 PM] ERROR: Errors.EMPTY_QUEUE
ERROR:btrack.utils:ERROR: Errors.EMPTY_QUEUE
[INFO]

No objects to track.


In [ ]:
# rolling ball background subtraction or flat-field correction for background correction.
#apply it before the segmentation of nuclei
# Specify rolling ball radius
rolling_ball_radius = 50  # Adjust this value based on your specific images

# Import necessary functions
from skimage.morphology import white_tophat, ball

# ...

## Segment nuclei
# Loading the StarDist2D model
modelStar = StarDist2D.from_pretrained('2D_versatile_fluo')

# Define structuring element for background subtraction
selem = ball(rolling_ball_radius)

# Loop over timepoints and run StarDist, extract x,y pos information
nuc_list = []
df = []
for i in tqdm(range(imERKl.shape[0])):
    # Apply rolling ball background subtraction
    img_background = white_tophat(imERKl[i, nuc_channel, ...], selem)

    # Subtract background from image
    img_corrected = imERKl[i, nuc_channel, ...] - img_background

    # Normalize the image
    img_norm = normalize(img_corrected, 1, 99.8, axis=0)  # Use axis=0 for normalization

    # Run StarDist and get label and detail information
    label = []
    detail = []
    for axis in 'YX':
        l, d = modelStar.predict_instances(img_norm, prob_thresh=0.47, show_tile_progress=False, verbose=False, axes=axis)
        label.append(l)
        detail.append(d)

    # Calculate regionprops and create DataFrame
    df_class = skimage.measure.regionprops_table(label[0], intensity_image=imERKl[i, nuc_channel, ...],
                                                 properties=('label', 'centroid', 'area'))
    df_class = pd.DataFrame(df_class)
    df_class['t'] = i
    df_class['centroid-0'] = df_class['centroid-0'].astype('int')
    df_class['centroid-1'] = df_class['centroid-1'].astype('int')
    df_class.rename(columns={'centroid-0': 'y', 'centroid-1': 'x'}, inplace=True)

    # Append label to the list
    nuc_list.append(label[0])

    # Append DataFrame to the list
    df.append(df_class)

# Concatenate the DataFrames
df = pd.concat(df)

# Stack the labels
seg_stack = np.stack(nuc_list)


In [ ]:
#for image stabilization depends on whether am dealing with translational shifts, rotational shifts,
# or more complex deformations
#but for complex deformations
## Load images
# ...

# Make sure images are loaded as floating point numbers
imERKl = imERKl.astype(float)

## Image stabilization
from skimage.registration import PiecewiseAffineTransform, warp

# Initialize array to hold stabilized images
imERKl_stabilized = np.empty_like(imERKl)

# Set first stabilized image as the first original image
imERKl_stabilized[0] = imERKl[0]

# Register each image to the previous one
for i in range(1, imERKl.shape[0]):
    # Estimate piecewise affine transformation from previous to current image
    transformation = PiecewiseAffineTransform()
    transformation.estimate(imERKl[i-1], imERKl[i])

    # Apply transformation to current image
    imERKl_stabilized[i] = warp(imERKl[i], transformation)

# Use stabilized images for the rest of the pipeline
imERKl = imERKl_stabilized
